In [ ]:
# BIDDER EVALUATION SYSTEM - Google Colab Version

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

# Step 1: Generate Sample Bidder Data
np.random.seed(42)

bidders = [f"Bidder_{i+1}" for i in range(15)]
data = {
    "Bidder": bidders,
    "Cost": np.random.randint(100000, 500000, size=len(bidders)),
    "Experience": np.random.randint(1, 10, size=len(bidders)),  # in years
    "QualityScore": np.random.randint(60, 100, size=len(bidders)),  # out of 100
    "DeliveryTime": np.random.randint(15, 60, size=len(bidders))  # in days
}

df = pd.DataFrame(data)
print("=== Raw Bidder Data ===")
print(df)

# Step 2: Define Evaluation Weights
# Lower cost and delivery time is better. Higher experience and quality is better.
weights = {
    "Cost": 0.4,
    "Experience": 0.2,
    "QualityScore": 0.3,
    "DeliveryTime": 0.1
}

print("\n=== Evaluation Criteria Weights ===")
print(weights)

# Step 3: Normalize Data for Fair Comparison
df_normalized = df.copy()

# Normalize cost and delivery time by min-max (lower is better → inverse)
df_normalized["Cost"] = 1 - (df["Cost"] - df["Cost"].min()) / (df["Cost"].max() - df["Cost"].min())
df_normalized["DeliveryTime"] = 1 - (df["DeliveryTime"] - df["DeliveryTime"].min()) / (df["DeliveryTime"].max() - df["DeliveryTime"].min())

# Normalize experience and quality (higher is better)
df_normalized["Experience"] = (df["Experience"] - df["Experience"].min()) / (df["Experience"].max() - df["Experience"].min())
df_normalized["QualityScore"] = (df["QualityScore"] - df["QualityScore"].min()) / (df["QualityScore"].max() - df["QualityScore"].min())

print("\n=== Normalized Data ===")
print(df_normalized)

# Step 4: Weighted Scoring
def calculate_score(row, weights):
    score = 0
    for k in weights:
        score += row[k] * weights[k]
    return score

df_normalized["Score"] = df_normalized.apply(lambda row: calculate_score(row, weights), axis=1)

# Step 5: Rank Bidders
df_final = df.copy()
df_final["Score"] = df_normalized["Score"]
df_final["Rank"] = df_final["Score"].rank(ascending=False).astype(int)
df_final = df_final.sort_values(by="Score", ascending=False).reset_index(drop=True)

print("\n=== Final Evaluation Results ===")
print(tabulate(df_final, headers='keys', tablefmt='fancy_grid', showindex=False))

# Step 6: Visualizations
plt.figure(figsize=(14, 6))
sns.barplot(x='Bidder', y='Score', data=df_final, palette='viridis')
plt.title("Bidder Evaluation Scores")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 6))
sns.heatmap(df_normalized.set_index("Bidder")[["Cost", "Experience", "QualityScore", "DeliveryTime"]], annot=True, cmap='coolwarm')
plt.title("Normalized Bidder Attributes")
plt.show()

# Step 7: Show Best Bidder
best_bidder = df_final.iloc[0]
print(f"\n🏆 Best Bidder: {best_bidder['Bidder']} with Score: {best_bidder['Score']:.3f}")

# Step 8: Export to Excel (if desired)
df_final.to_csv("bidder_evaluation_results.csv", index=False)
print("\n✅ Evaluation complete. Results saved to 'bidder_evaluation_results.csv'")


In [ ]:
"""
Diploma Document OCR and Configuration Suite

This extended Jupyter notebook provides a complete, modular, and scalable OCR-based document
processing pipeline tailored for high-volume, multi-page diploma or academic document processing.
It includes advanced configuration options, metadata entry, batch processing support, page-level
navigation, similarity evaluation, and validation mechanisms to ensure extracted data is both accurate
and properly attributed.

Primary Features:
- Accepts long, high-resolution multi-page documents (PDF or image)
- Supports multiple OCR engines: Tesseract, DocTR (TensorFlow, PyTorch)
- Allows metadata entry: university name, degree type, graduate name, issue date, etc.
- Provides structured output for downstream validation or archival
- Built-in interface for comparing OCR results to ground-truth manually entered by operator
- Optional export to structured JSON for further processing (e.g., database or backend validation)
"""

import pytesseract
import fitz  # PyMuPDF
from doctr.io import DocumentFile
from doctr.models import ocr_predictor
from PIL import Image
from IPython.display import display, clear_output
import ipywidgets as widgets
import io
import json
from difflib import SequenceMatcher

# Section: Upload interface
upload_widget = widgets.FileUpload(accept='.pdf,.png,.jpg,.jpeg', multiple=False)
display(widgets.HTML("<h2>Diploma Document Upload</h2>"))
display(upload_widget)

# Section: OCR Engine Selection
ocr_selector = widgets.Dropdown(
    options=['Tesseract', 'DocTR (TensorFlow)', 'DocTR (PyTorch)'],
    value='DocTR (TensorFlow)',
    description='OCR Engine:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)
display(widgets.HTML("<h3>Select OCR Engine</h3>"))
display(ocr_selector)

# Section: Diploma Metadata Configuration
metadata_fields = {
    'university_name': widgets.Text(description='University Name:'),
    'degree_type': widgets.Text(description='Degree Type:'),
    'graduate_name': widgets.Text(description='Graduate Name:'),
    'graduation_date': widgets.Text(description='Graduation Date:'),
    'document_id': widgets.Text(description='Document ID (Optional):')
}
display(widgets.HTML("<h3>Diploma Metadata Entry</h3>"))
for field in metadata_fields.values():
    display(field)

# OCR Execution Function
def perform_ocr(file, engine_choice):
    extracted_text_pages = []
    if file.name.endswith('.pdf'):
        doc = fitz.open(stream=file.content, filetype="pdf")
        images = [Image.open(io.BytesIO(page.get_pixmap(dpi=300).tobytes())) for page in doc]
    else:
        images = [Image.open(io.BytesIO(file.content))]

    if engine_choice == 'Tesseract':
        for image in images:
            text = pytesseract.image_to_string(image)
            extracted_text_pages.append(text)
    else:
        doc_input = DocumentFile.from_pdf(io.BytesIO(file.content)) if file.name.endswith('.pdf') else DocumentFile.from_images(images)
        model = ocr_predictor(pretrained=True, assume_straight_pages=True, use_pytorch=(engine_choice == 'DocTR (PyTorch)'))
        result = model(doc_input)
        for page in result.pages:
            text = '\n'.join([block['value'] for block in page.blocks])
            extracted_text_pages.append(text)

    return extracted_text_pages

# Page Navigator

def show_page_navigation(pages):
    page_selector = widgets.Dropdown(
        options=[(f"Page {i+1}", i) for i in range(len(pages))],
        description="Select Page:"
    )
    output_area = widgets.Output()

    def on_page_change(change):
        with output_area:
            clear_output()
            print(pages[change['new']])

    page_selector.observe(on_page_change, names='value')
    display(widgets.HTML("<h3>Page Navigator</h3>"))
    display(page_selector)
    display(output_area)
    on_page_change({'new': 0})

# Similarity Checker Interface
def show_evaluation_interface(ocr_pages):
    expected_textarea = widgets.Textarea(
        value='',
        placeholder='Paste the expected ground-truth result here for validation...',
        layout=widgets.Layout(width='100%', height='150px')
    )
    eval_button = widgets.Button(description="Evaluate OCR Similarity")
    eval_output = widgets.Output()

    def on_eval_clicked(b):
        with eval_output:
            clear_output()
            combined_ocr_text = '\n'.join(ocr_pages)
            ratio = SequenceMatcher(None, expected_textarea.value.strip(), combined_ocr_text.strip()).ratio()
            print(f"OCR Similarity: {ratio:.2%}")

    eval_button.on_click(on_eval_clicked)
    display(widgets.HTML("<h3>OCR Evaluation</h3>"))
    display(expected_textarea)
    display(eval_button)
    display(eval_output)

# Export Metadata + OCR to JSON

def export_to_json(metadata, ocr_text):
    payload = {
        'metadata': {k: v.value for k, v in metadata.items()},
        'ocr_text': ocr_text
    }
    print("\nStructured Export:")
    print(json.dumps(payload, indent=4))

export_btn = widgets.Button(description="Export to JSON")

# Bind export
if upload_widget.value:
    file_data = list(upload_widget.value.values())[0]
    engine_selected = ocr_selector.value
    display(widgets.HTML("<h3>Running OCR - Please Wait</h3>"))
    ocr_results = perform_ocr(file_data, engine_selected)
    display(widgets.HTML("<h3>OCR Completed</h3>"))
    show_page_navigation(ocr_results)
    show_evaluation_interface(ocr_results)

    def on_export_clicked(b):
        export_to_json(metadata_fields, ocr_results)

    export_btn.on_click(on_export_clicked)
    display(export_btn)
else:
    display(widgets.HTML("<b>Please upload a document above to begin processing.</b>"))

In [ ]:
# COLAB CV CONFIGURATION: HUGE MULTI-MODEL INITIALIZER

# Step 1: Install and Import Dependencies
!pip install -q timm torchvision transformers datasets opencv-python
import os
import cv2
import torch
import torchvision
import timm
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from torchvision import transforms
from tensorflow.keras.applications import *
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
from PIL import Image

print("All dependencies imported!")

# Step 2: Load and Preprocess a Sample Image
def load_sample_image(path='sample.jpg'):
    # Download a sample image
    if not os.path.exists(path):
        !wget -q https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/YellowLabradorLooking_new.jpg/640px-YellowLabradorLooking_new.jpg -O sample.jpg
    img = Image.open(path).convert('RGB')
    return img

img = load_sample_image()
plt.imshow(img)
plt.title("Sample Input Image")
plt.axis('off')
plt.show()

# Resize for all models
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
input_tensor = transform(img).unsqueeze(0)

# Step 3: TensorFlow / Keras Models
keras_models = {
    "VGG16": VGG16(weights='imagenet', include_top=False),
    "VGG19": VGG19(weights='imagenet', include_top=False),
    "ResNet50": ResNet50(weights='imagenet', include_top=False),
    "ResNet101": ResNet101(weights='imagenet', include_top=False),
    "InceptionV3": InceptionV3(weights='imagenet', include_top=False),
    "Xception": Xception(weights='imagenet', include_top=False),
    "MobileNet": MobileNet(weights='imagenet', include_top=False),
    "DenseNet201": DenseNet201(weights='imagenet', include_top=False),
    "NASNetMobile": NASNetMobile(weights='imagenet', include_top=False),
    "EfficientNetB7": EfficientNetB7(weights='imagenet', include_top=False)
}

def keras_model_summary():
    for name, base_model in keras_models.items():
        model = tf.keras.Sequential([
            base_model,
            GlobalAveragePooling2D(),
            Dense(1024, activation='relu'),
            Dense(10, activation='softmax')
        ])
        print(f"Summary of {name}:")
        model.build((None, 224, 224, 3))
        model.summary()
        print("="*80)

keras_model_summary()

# Step 4: PyTorch Vision Models
torch_models = {
    "resnet18": torchvision.models.resnet18(pretrained=True),
    "resnet50": torchvision.models.resnet50(pretrained=True),
    "alexnet": torchvision.models.alexnet(pretrained=True),
    "squeezenet": torchvision.models.squeezenet1_0(pretrained=True),
    "vgg16": torchvision.models.vgg16(pretrained=True),
    "densenet": torchvision.models.densenet161(pretrained=True),
    "inception": torchvision.models.inception_v3(pretrained=True, aux_logits=False),
    "googlenet": torchvision.models.googlenet(pretrained=True),
    "shufflenet": torchvision.models.shufflenet_v2_x1_0(pretrained=True),
    "mobilenet": torchvision.models.mobilenet_v2(pretrained=True),
    "resnext": torchvision.models.resnext50_32x4d(pretrained=True),
    "wide_resnet": torchvision.models.wide_resnet50_2(pretrained=True),
    "mnasnet": torchvision.models.mnasnet1_0(pretrained=True),
}

for name, model in torch_models.items():
    model.eval()
    out = model(input_tensor)
    print(f"{name} output shape: {out.shape}")

# Step 5: TIMM Models
timm_models = [
    'vit_base_patch16_224',
    'swin_base_patch4_window7_224',
    'efficientnet_b3a',
    'resnext101_32x8d',
    'regnety_160',
    'tf_efficientnet_b7_ns',
    'convnext_base',
    'beit_base_patch16_224'
]

print("TIMM models output shapes:")
for model_name in timm_models:
    model = timm.create_model(model_name, pretrained=True)
    model.eval()
    out = model(input_tensor)
    print(f"{model_name}: {out.shape}")

# Step 6: Transformers Vision Models
hf_models = [
    "google/vit-base-patch16-224",
    "microsoft/resnet-50",
    "facebook/deit-base-distilled-patch16-224",
    "microsoft/swin-tiny-patch4-window7-224"
]

for model_name in hf_models:
    print(f"Loading HuggingFace model: {model_name}")
    extractor = AutoFeatureExtractor.from_pretrained(model_name)
    model = AutoModelForImageClassification.from_pretrained(model_name)

    img_array = extractor(images=img, return_tensors="pt")
    with torch.no_grad():
        output = model(**img_array)
    print(f"{model_name} logits shape: {output.logits.shape}")

# Step 7: Sample CV Tasks
print("\nPerforming Sample Classification with ResNet18:")
resnet18 = torchvision.models.resnet18(pretrained=True)
resnet18.eval()
output = resnet18(input_tensor)
prob = torch.nn.functional.softmax(output[0], dim=0)
top5 = torch.topk(prob, 5)

# Download labels
!wget -q https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

for i in range(5):
    print(f"{categories[top5.indices[i]]}: {top5.values[i].item()*100:.2f}%")

# Step 8: Object Detection with Pretrained Faster R-CNN
print("\nRunning object detection with Faster R-CNN")
od_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
od_model.eval()
image_tensor = transform(img).unsqueeze(0)
predictions = od_model(image_tensor)

for i in range(len(predictions[0]['boxes'])):
    score = predictions[0]['scores'][i].item()
    if score > 0.5:
        box = predictions[0]['boxes'][i].detach().numpy()
        print(f"Object {i}: Box={box}, Score={score:.2f}")

# Step 9: Semantic Segmentation with DeepLabV3
print("\nRunning semantic segmentation with DeepLabV3")
seg_model = torchvision.models.segmentation.deeplabv3_resnet101(pretrained=True)
seg_model.eval()
output = seg_model(image_tensor)['out']
seg = output.squeeze().argmax(0).detach().cpu().numpy()

plt.imshow(seg)
plt.title("Segmentation Output")
plt.axis("off")
plt.show()

print("\nDone. You have configured a huge number of CV models!")


In [ ]:
# BIDDER EVALUATION SYSTEM - Google Colab Version

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

# Step 1: Generate Sample Bidder Data
np.random.seed(42)

bidders = [f"Bidder_{i+1}" for i in range(15)]
data = {
    "Bidder": bidders,
    "Cost": np.random.randint(100000, 500000, size=len(bidders)),
    "Experience": np.random.randint(1, 10, size=len(bidders)),  # in years
    "QualityScore": np.random.randint(60, 100, size=len(bidders)),  # out of 100
    "DeliveryTime": np.random.randint(15, 60, size=len(bidders))  # in days
}

df = pd.DataFrame(data)
print("=== Raw Bidder Data ===")
print(df)

# Step 2: Define Evaluation Weights
# Lower cost and delivery time is better. Higher experience and quality is better.
weights = {
    "Cost": 0.4,
    "Experience": 0.2,
    "QualityScore": 0.3,
    "DeliveryTime": 0.1
}

print("\n=== Evaluation Criteria Weights ===")
print(weights)

# Step 3: Normalize Data for Fair Comparison
df_normalized = df.copy()

# Normalize cost and delivery time by min-max (lower is better → inverse)
df_normalized["Cost"] = 1 - (df["Cost"] - df["Cost"].min()) / (df["Cost"].max() - df["Cost"].min())
df_normalized["DeliveryTime"] = 1 - (df["DeliveryTime"] - df["DeliveryTime"].min()) / (df["DeliveryTime"].max() - df["DeliveryTime"].min())

# Normalize experience and quality (higher is better)
df_normalized["Experience"] = (df["Experience"] - df["Experience"].min()) / (df["Experience"].max() - df["Experience"].min())
df_normalized["QualityScore"] = (df["QualityScore"] - df["QualityScore"].min()) / (df["QualityScore"].max() - df["QualityScore"].min())

print("\n=== Normalized Data ===")
print(df_normalized)

# Step 4: Weighted Scoring
def calculate_score(row, weights):
    score = 0
    for k in weights:
        score += row[k] * weights[k]
    return score

df_normalized["Score"] = df_normalized.apply(lambda row: calculate_score(row, weights), axis=1)

# Step 5: Rank Bidders
df_final = df.copy()
df_final["Score"] = df_normalized["Score"]
df_final["Rank"] = df_final["Score"].rank(ascending=False).astype(int)
df_final = df_final.sort_values(by="Score", ascending=False).reset_index(drop=True)

print("\n=== Final Evaluation Results ===")
print(tabulate(df_final, headers='keys', tablefmt='fancy_grid', showindex=False))

# Step 6: Visualizations
plt.figure(figsize=(14, 6))
sns.barplot(x='Bidder', y='Score', data=df_final, palette='viridis')
plt.title("Bidder Evaluation Scores")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 6))
sns.heatmap(df_normalized.set_index("Bidder")[["Cost", "Experience", "QualityScore", "DeliveryTime"]], annot=True, cmap='coolwarm')
plt.title("Normalized Bidder Attributes")
plt.show()

# Step 7: Show Best Bidder
best_bidder = df_final.iloc[0]
print(f"\n🏆 Best Bidder: {best_bidder['Bidder']} with Score: {best_bidder['Score']:.3f}")

# Step 8: Export to Excel (if desired)
df_final.to_csv("bidder_evaluation_results.csv", index=False)
print("\n✅ Evaluation complete. Results saved to 'bidder_evaluation_results.csv'")


In [ ]:
# BIDDER EVALUATION SYSTEM - Google Colab Version

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

# Step 1: Generate Sample Bidder Data
np.random.seed(42)

bidders = [f"Bidder_{i+1}" for i in range(15)]
data = {
    "Bidder": bidders,
    "Cost": np.random.randint(100000, 500000, size=len(bidders)),
    "Experience": np.random.randint(1, 10, size=len(bidders)),  # in years
    "QualityScore": np.random.randint(60, 100, size=len(bidders)),  # out of 100
    "DeliveryTime": np.random.randint(15, 60, size=len(bidders))  # in days
}

df = pd.DataFrame(data)
print("=== Raw Bidder Data ===")
print(df)

# Step 2: Define Evaluation Weights
# Lower cost and delivery time is better. Higher experience and quality is better.
weights = {
    "Cost": 0.4,
    "Experience": 0.2,
    "QualityScore": 0.3,
    "DeliveryTime": 0.1
}

print("\n=== Evaluation Criteria Weights ===")
print(weights)

# Step 3: Normalize Data for Fair Comparison
df_normalized = df.copy()

# Normalize cost and delivery time by min-max (lower is better → inverse)
df_normalized["Cost"] = 1 - (df["Cost"] - df["Cost"].min()) / (df["Cost"].max() - df["Cost"].min())
df_normalized["DeliveryTime"] = 1 - (df["DeliveryTime"] - df["DeliveryTime"].min()) / (df["DeliveryTime"].max() - df["DeliveryTime"].min())

# Normalize experience and quality (higher is better)
df_normalized["Experience"] = (df["Experience"] - df["Experience"].min()) / (df["Experience"].max() - df["Experience"].min())
df_normalized["QualityScore"] = (df["QualityScore"] - df["QualityScore"].min()) / (df["QualityScore"].max() - df["QualityScore"].min())

print("\n=== Normalized Data ===")
print(df_normalized)

# Step 4: Weighted Scoring
def calculate_score(row, weights):
    score = 0
    for k in weights:
        score += row[k] * weights[k]
    return score

df_normalized["Score"] = df_normalized.apply(lambda row: calculate_score(row, weights), axis=1)

# Step 5: Rank Bidders
df_final = df.copy()
df_final["Score"] = df_normalized["Score"]
df_final["Rank"] = df_final["Score"].rank(ascending=False).astype(int)
df_final = df_final.sort_values(by="Score", ascending=False).reset_index(drop=True)

print("\n=== Final Evaluation Results ===")
print(tabulate(df_final, headers='keys', tablefmt='fancy_grid', showindex=False))

# Step 6: Visualizations
plt.figure(figsize=(14, 6))
sns.barplot(x='Bidder', y='Score', data=df_final, palette='viridis')
plt.title("Bidder Evaluation Scores")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 6))
sns.heatmap(df_normalized.set_index("Bidder")[["Cost", "Experience", "QualityScore", "DeliveryTime"]], annot=True, cmap='coolwarm')
plt.title("Normalized Bidder Attributes")
plt.show()

# Step 7: Show Best Bidder
best_bidder = df_final.iloc[0]
print(f"\n🏆 Best Bidder: {best_bidder['Bidder']} with Score: {best_bidder['Score']:.3f}")

# Step 8: Export to Excel (if desired)
df_final.to_csv("bidder_evaluation_results.csv", index=False)
print("\n✅ Evaluation complete. Results saved to 'bidder_evaluation_results.csv'")


In [ ]:
# BIDDER EVALUATION SYSTEM - Google Colab Version

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

# Step 1: Generate Sample Bidder Data
np.random.seed(42)

bidders = [f"Bidder_{i+1}" for i in range(15)]
data = {
    "Bidder": bidders,
    "Cost": np.random.randint(100000, 500000, size=len(bidders)),
    "Experience": np.random.randint(1, 10, size=len(bidders)),  # in years
    "QualityScore": np.random.randint(60, 100, size=len(bidders)),  # out of 100
    "DeliveryTime": np.random.randint(15, 60, size=len(bidders))  # in days
}

df = pd.DataFrame(data)
print("=== Raw Bidder Data ===")
print(df)

# Step 2: Define Evaluation Weights
# Lower cost and delivery time is better. Higher experience and quality is better.
weights = {
    "Cost": 0.4,
    "Experience": 0.2,
    "QualityScore": 0.3,
    "DeliveryTime": 0.1
}

print("\n=== Evaluation Criteria Weights ===")
print(weights)

# Step 3: Normalize Data for Fair Comparison
df_normalized = df.copy()

# Normalize cost and delivery time by min-max (lower is better → inverse)
df_normalized["Cost"] = 1 - (df["Cost"] - df["Cost"].min()) / (df["Cost"].max() - df["Cost"].min())
df_normalized["DeliveryTime"] = 1 - (df["DeliveryTime"] - df["DeliveryTime"].min()) / (df["DeliveryTime"].max() - df["DeliveryTime"].min())

# Normalize experience and quality (higher is better)
df_normalized["Experience"] = (df["Experience"] - df["Experience"].min()) / (df["Experience"].max() - df["Experience"].min())
df_normalized["QualityScore"] = (df["QualityScore"] - df["QualityScore"].min()) / (df["QualityScore"].max() - df["QualityScore"].min())

print("\n=== Normalized Data ===")
print(df_normalized)

# Step 4: Weighted Scoring
def calculate_score(row, weights):
    score = 0
    for k in weights:
        score += row[k] * weights[k]
    return score

df_normalized["Score"] = df_normalized.apply(lambda row: calculate_score(row, weights), axis=1)

# Step 5: Rank Bidders
df_final = df.copy()
df_final["Score"] = df_normalized["Score"]
df_final["Rank"] = df_final["Score"].rank(ascending=False).astype(int)
df_final = df_final.sort_values(by="Score", ascending=False).reset_index(drop=True)

print("\n=== Final Evaluation Results ===")
print(tabulate(df_final, headers='keys', tablefmt='fancy_grid', showindex=False))

# Step 6: Visualizations
plt.figure(figsize=(14, 6))
sns.barplot(x='Bidder', y='Score', data=df_final, palette='viridis')
plt.title("Bidder Evaluation Scores")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 6))
sns.heatmap(df_normalized.set_index("Bidder")[["Cost", "Experience", "QualityScore", "DeliveryTime"]], annot=True, cmap='coolwarm')
plt.title("Normalized Bidder Attributes")
plt.show()

# Step 7: Show Best Bidder
best_bidder = df_final.iloc[0]
print(f"\n🏆 Best Bidder: {best_bidder['Bidder']} with Score: {best_bidder['Score']:.3f}")

# Step 8: Export to Excel (if desired)
df_final.to_csv("bidder_evaluation_results.csv", index=False)
print("\n✅ Evaluation complete. Results saved to 'bidder_evaluation_results.csv'")
